In [9]:
import cv2
import os
import time
import json
import random
from PIL import Image
import numpy as np
from mss import mss
import serial, time
import binascii
import imageio
import urllib
import requests

In [2]:
cap = cv2.VideoCapture(0)
sct = mss()

In [16]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
src_dev = 2
w, h = 64, 64
monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}

In [15]:
gifurl = 'https://gifs.iloopit.net/resources/1ead57a3-48e4-4fb1-86ea-07527a7c079a/converted.gif'
fname = 'tmp.gif'

## Read the gif from the web, save to the disk
if not os.path.exists(fname):
    imdata = urllib.request.urlopen(gifurl).read()
    imbytes = bytearray(imdata)
    open(fname,"wb+").write(imdata)

gif = imageio.mimread(fname)
num_frames = len(gif)
print("Total {} frames in the gif!".format(nums))

Total 108 frames in the gif!


In [14]:
imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in gif]
imgs = [cv2.resize(rgb, (64, 64)) for rgb in imgs]
imgs = [cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1] for img in imgs]

## Display the gif
i = 0

while True:
    cv2.imshow("gif", imgs[i])
    if cv2.waitKey(100)&0xFF == 27:
        break
    i = (i+1)%nums
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
cframe = 0
while True:
    while True:
        rline = arduino.readline().strip()
        if len(rline) > 0:
            print(rline)
        else:
            break
        
    if src_dev == 0:
        ret, frame = cap.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif src_dev == 1:
        rgb = np.array(sct.grab(monitor))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    elif src_dev == 2:
        rgb = imgs[cframe]
        cframe = (cframe + 20) % num_frames
    else:
        raise ValueError('wtf')
        

    obuff=cv2.resize(rgb, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    #print(obuff)
    #break
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    #print(binascii.hexlify(nobuff))

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64

        ln = arduino.write(sbuff)
        arduino.flush()
        print('%s, %s' % (ln, binascii.hexlify(sbuff)))
        time.sleep(0.15)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(1)
    #time.sleep(2)
    #break


65, b'00ffff3f80ffff0ffdff073ec0ffffcffeffff3fe0ffff07fdffff1ff0ffff07feffff0ff8ffffc7feffff07f8ffff07feffff07f8ffff03ffffff03fcffff03ff'
65, b'01ffff037cfcff07feffff0178f8ff0fffffff017cf8ff0efeffff0138f81f16feffff003cf0830bfeffff003cf8f00ffeffff007cf8fe0fffffff00fefcff07ff'
65, b'02ff7f00fefdff0fffff7f00fcfdff87ffff7f00fcd3ff87ffff7f00fcd2ff87fffe7f00fdc0ff87ff0800003fc4ff87ffcb41800fc8ff87ffff7f001cccff87ff'
65, b'03ff7f001ec0ff87ffff7fb01ec0ff83ffff7f801e00ff87ffff7f000e00f887ffff7f000e00e0c7ffff7f000f0000c0ffff7f8007000080ffff7f8007000080ff'
65, b'04ff7f8003000080ffff7f8007000080ffff7fd007000080ffff3ff00b400080ffff3ff003e00080ffff3ff003f00380ffff3ff801f00f80ffff1ff801f01f80ff'
65, b'05ff1ff800f87f80ffff1ffc00f8ff81ffff1ffe00fcff03ffff0f9e80ffff07ffff0f5f80ffff07ffff070780ffff07ffff070380ffff07ffff8701c0ffff07fe'
65, b'06ff0700c0fcff07feff0700c0fcff07feff030080ffff07feff030000ffff07feff030000ffff07feff0b0000ffff07feff050000ffff0ffcff070000ffff1ffc'
65, b'07ff030000ffff1ff8ff0100008f

65, b'04ff3fe0000000f0ffff3fc0030000f0ffff1fe0010000f0ffff1fe8012000f0ffff1ff801f000e0ffff0ff800f801f0ffff0fbc00f807f0ffff0ffc00f80fe0ff'
65, b'05ff0f7c00fc3fe0ffff072e00fcffe0ffff070700feffe1ffff4303c0ffffe1ffff8301c0ffffe1ffff4100e0ffffe1ffff0100e0ffffe1ffff010020ffffe1ff'
65, b'06ff010060ffffe1ffff0000e0feffe1ffff0400c0ffffe1ff7f0300e0ffffc1ffff0000c0ffffc1ffff0000c0ffffc3ffff0000c0fdffc3ffff0000c0ffffc7ff'
65, b'077f000080ffff8fff7f0000c0ffff9fff7f000080ffffffff7f000080ffbfeefb7f000000ff7f7e8f7f000000f27feadb7f000000f2ffffff7f000000f2ffffff'
65, b'00ffffff01fcff7fe0ffffff80ffffffe0ffff3f80ffff7ff0ff077ec0ffffbff3ffff3fe0ffff7fe0ffff1fe0fffffff0ffff1fe0ffffffe0ffff0fe0e1ff7ff0'
65, b'01ffff0fe0e1ff7fe0ffff0ff0e1ffefe0ffff07f0a1ffc3f0ffff07f0e0ff70f0ffff07f0e11f7ef8ffff07f0f1877ff8ffff03f8e7f17ff8ffff03f0fffd7ff0'
65, b'02ffff03f0a7ff7ff8ffff03f85bfe7ff8ffff03f803fe7ff8ffff03fc2bff7ff8ffff01f521ff7ffcffff817e00ff3ffcfeff817c00ff3ffc200080b800fe3ffc'
65, b'0327f381ba00fe3ffcffff833b00

65, b'00f0ffff033cfefff9fcffff01bffffff8fcffff80fffffff2fcffff80ffff3ff0f0ff3fc0ffff3ff285ff3fe0ffffbff0ff0017e0ffff7ff0ffff1fe0f3ff7ff0'
65, b'01ffff1fe0e1ff7ff0ffff0fe0e3ffbff8ffff0fe0e1ff7ff8ffff07f0e1ff1ff8ffff07f0e0ff47f8ffff07f081ff70fcffff07f0e33f7cfcffff07f0f70f7ffc'
65, b'02ffff03f0cfe37ffcffff03f087f87ffcffff03f09bfc7ffcffff03fc13fe7ffcffff03f221ff7ffcffff83fc21ff7ffcffff83fc81ff3ffcffff817811ff3ffc'
65, b'03ffff81f800ff3ffcffff817b00fe3ffcc00002bb00f03ffe40c6037800003ffcffff0338080000fcffff033e000000f8ffff031c000000f8ffff039e000000f8'
65, b'04ffff033e000000f8ffff019e000000f8ffffc17f000000f8ffffc17f800100f8ffffc0ff800f00f8ffffc01f803f00f8ffffc01fc0ff00f0ffffc00fc0ff01d0'
65, b'05ffffe007e0ff0ff8ffffe007e0ff3ff8ff7ff003e0ff3ff0ff7f7800f0ff3ff0ff3f3800feff3ff0ff3f1c00feff3ff0ff3f0400feff3ff0ff3f0000ffff3ff0'
65, b'06ff3f0000ffff3ff0ff1f0000ffff3fe0ff5f0000f5ff7fe0ff3f0000eeff7fe0ff1f0000fdff7fc0ff0f0000fdffffc0ff0f0000feffffe0ff1f0000fcffffc1'
65, b'07ff0f0000fcffffa3ff070000fc

65, b'04ffff031c0000e0ffffff011c0000e0ffffff011c0000e0ffffff011c0000e0ffffff411c0000e0ffffffc01e0002e0ffffffc0068007e0ffffffc00f801fc0ff'
65, b'05ffffc00f803fc0ffff7fc007c0ffc0ffff7fe003c0ffc1ffff7f6006e0ffc1ffff3f3000fcffc3ffff1f1c00feffc3ffff1f0800feffc3ffff1f0200feffc3ff'
65, b'06ff1f0000feff83ffff0f0000faff81ffff0f0000ffff83ffff470000ffff83ffff270000ffff83ffff330000ffff83ffff130000fdff83ffff1f0000fdff87ff'
65, b'07ff0d0000fcff8fffff070000fcff1fffff070000fcffffffff030000fcbfeefbff030000fc7f7e8fff070000f87feadbff030000f0fffffaff030000d0ffffff'
65, b'00ff87a601ffffc1ffffff7f00feffc1ffffff7f80ffffd0ffffff7fc0ffffd0ffffff3fe0ffffd4ffffff1fe0ffffe0ffffff0ff0ffffe4ffffff0ff0ffffe0ff'
65, b'01ffff07f0fbffe0ffffff07f0f1ffe1ffffff03f0f1bfe1ffffff03f0e18fe1ffffff03f0e0e1e1ffffff01f8e0f8f1ffffff01f860ffe1fffff701f8e1fff1ff'
65, b'02ffff01f8e3fff1ffffff01f8e7fff9ffffff00f0b7fff9ffffff00f093fef9fff80500f001fef9ff080000f021fff9ffefff00f400fff9ffffff00bc00fff9ff'
65, b'03ffff003e00fff9ffffff403800

65, b'00fffcff3ff8ff3fe0bffeff0ff0ff3ff0fffdff07fcff3ff07ff87f02feff3ff07f00f202ffff3ff0ffffff01ffff3fc0ffffff80ffff3fe0ffffff808fff7fc0'
65, b'01ffff7f808fff7fe0ffff7f808fff3ff0ffff3f800fff0fe0ffff3f8007ff63f0ffff1f800f7f78f0ffff1f80071f7ef0ffff1f801fc77ff0ffff1f807fff7ff0'
65, b'02ffff1f80fff57ff0ffff0f80fff57ff8ffff0f80fff17ff8ffff0f80fff87ff8ffff0f80bffa7ff8ffff0fa01ffc7ff8ffff0fe00ffc7ff8ffff0fe00ff07ff8'
65, b'03200008a40ff23ff801830fc403e27ff8ffff0fa40f003ff8ffff0f90010100f8ffff0f98020000f8ffff0f80020000f0ffff0f80070000f8ffff1fc0010000f8'
65, b'04ffff1fe0000000f8ffff1fe0000000f8ffff0ff0000000f0ffff0fe0010000f0ffff0fe0013000f0ffff0fe001f800f0ffff0fe200f801e0ffff076600fc07e0'
65, b'05ffff077e00fc1ff0ffff076600fc3fe0ffff070e00fc7fe0ffff030300fe7fe0ffff030180ff7fe0ffff8100e0ff7fe0ffff0100f0ff7fe0ffff0000f0ff7fe0'
65, b'06ffff0000f0ff7fc0ffff0000f0ff7fc0ff7f0000d0ff7fc0ff7f0000c0ff7fc0ff3f0100f8ffffc0ffbf0000f8ffffc0ff7f0000f8ffffc1ff7f0000f8ffff83'
65, b'07ff3f0000f0ffff87ff3f0000f0

In [ ]:
cv2.destroyAllWindows()

In [ ]:
#w, h = 200, 200
#monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
#img = Image.frombytes('RGB', (w,h), sct.grab(monitor).rgb)
#cv2.imshow('frame', np.array(img))
cv2.imshow('frame', np.array(sct.grab(monitor)))
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
cap.release()

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
buff=bytearray(65)

In [ ]:
arduino.write(b'\x04'+buff)

In [ ]:
for i in range(65):
    buff[i] = i

In [ ]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


In [ ]:
while True:
    print(arduino.readline())